In [23]:
##### connect to the elasticsearch server
import json
from pprint import pprint
import os
import time
import pandas as pd 
from datetime import datetime

import pathlib
from dotenv import load_dotenv
from elasticsearch import Elasticsearch

from DataProfile_JSON_db import *
load_dotenv()

##### dummy files to upload to minio and elasticsearch
path_to_main_input = "./examples/dummy_data"

class ESearch:
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.es = Elasticsearch(
            "http://localhost:9200", # deployed locally, no cloud
            basic_auth=(username, password))  # <-- connection options need to be added here
        client_info = self.es.info()
        print('Connected to Elasticsearch!')
        pprint(client_info.body)
        
    def create_index(self, index_name, metadata_profile):
        self.es.indices.create(index = index_name,  mappings = metadata_profile)

    def insert_document(self, document, index):
        return self.es.index(index = index, body=document)

##### print the client info, check if the elasticsearch server is running
my_es = ESearch("elastic", "genov4")


Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'SNT6DPXwT_SU_8AFROVQSw',
 'name': '75bdaad07a27',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2024-07-01T22:06:58.515911606Z',
             'build_flavor': 'default',
             'build_hash': '2afe7caceec8a26ff53817e5ed88235e90592a1b',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.10.0',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.14.2'}}


In [39]:
##### prepare data to be added to database
file_type = "bam"
input_files = [item for item in pathlib.Path(os.path.join(path_to_main_input, file_type)).glob("*.{}".format(file_type))]
input_metadata = pd.DataFrame(data = [item.name for item in input_files], columns = ["FileName"])
input_metadata["FileType"] = file_type
input_metadata["Labcode"] = input_metadata["FileName"].apply(lambda x: x.replace(".{}".format(file_type), ""))
input_metadata["path"] = ["./examples/dummy_data/{}/{}.{}".format(file_type, labcode, file_type) for labcode in input_metadata.Labcode.values]
input_metadata["project"] = "ECD"
input_metadata["sub_project"] = "ECD_read_based"
input_metadata["Date"] = datetime.now()
input_metadata["pipeline"] = "bismark_wgbs"

input_metadata = input_metadata.set_index("path")

input_metadata_dict = input_metadata.to_dict(orient = "index")

In [7]:
##### load the data profile database
db = DataProfileDB("ALL_DATA_PROFILES.json")

##### choose the data profile name, example: choose "dev2"
profile_name = "dev2"
my_es.create_index(index_name = profile_name, metadata_profile = db.get_record(profile_name)[profile_name])



ModuleNotFoundError: No module named 'Datetime'